In [4]:
!pip install torch numpy datasets scikit-learn pyplot

ERROR: Could not find a version that satisfies the requirement pyplot (from versions: none)
ERROR: No matching distribution found for pyplot


# Более оптимизированный метод

In [3]:
import json
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
import time
from tqdm import tqdm
import pickle
import os

# 1. Предобработка данных
def card_to_vector(card):
    """Преобразует карту (например, '14S') в вектор."""
    if not card or not isinstance(card, str):
        return np.zeros(5)
    try:
        nominal = int(card[:-1]) - 9  # 9->0, 10->1, ..., 14->5
        if nominal < 0 or nominal > 5:
            return np.zeros(5)
        suit = {'S': 0, 'C': 1, 'D': 2, 'H': 3}[card[-1]]
        vector = np.zeros(5)
        vector[0] = nominal
        vector[1 + suit] = 1
        return vector
    except (ValueError, KeyError):
        return np.zeros(5)

def state_to_vector(state):
    """Кодирует состояние игрока."""
    states = ['attack', 'defend', 'bat', 'pass', 'take']
    vector = np.zeros(len(states))
    if state in states:
        vector[states.index(state)] = 1
    return vector

def snapshot_to_input(parsed_data):
    """Преобразует предварительно разобранный снимок в вектор состояния."""
    try:
        # Проверяем наличие необходимых ключей
        if not isinstance(parsed_data, dict) or 'players' not in parsed_data or not parsed_data['players'] or 'trump' not in parsed_data:
            return None

        # Рука игрока
        player = next((p for p in parsed_data['players'] if p['id'] == parsed_data['players'][0]['id']), None)
        if player is None or 'hand' not in player:
            return None
        hand = np.zeros((24, 5))
        for i, card in enumerate(player.get('hand', [])[:24]):
            hand[i] = card_to_vector(card)

        # Стол
        table = np.zeros((6, 2, 5))  # 6 пар, атака+защита
        for i, pair in enumerate(parsed_data.get('table', [])[:6]):
            table[i, 0] = card_to_vector(pair.get('attack_card', {}).get('card', ''))
            table[i, 1] = card_to_vector(pair.get('defend_card', {}).get('card', ''))

        # Козырь
        trump = np.zeros(4)
        trump_suit = parsed_data.get('trump', 'S')[-1] if isinstance(parsed_data.get('trump'), str) else 'S'
        trump[{'S': 0, 'C': 1, 'D': 2, 'H': 3}.get(trump_suit, 0)] = 1

        # Состояние игрока
        state = state_to_vector(player.get('state', 'pass'))

        # Тип игры
        game_type = np.array([parsed_data.get('game_rules', {}).get('game_type', 0)])

        return np.concatenate([hand.flatten(), table.flatten(), trump, state, game_type])
    except Exception:
        return None

def extract_action(prev_data, curr_data):
    """Определяет действие между снимками."""
    try:
        player_id = prev_data['players'][0]['id']
        prev_table = prev_data.get('table', [])
        curr_table = curr_data.get('table', [])

        # Проверяем изменения на столе
        if len(curr_table) > len(prev_table):
            # Новая карта атаки
            new_pair = curr_table[-1]
            if (new_pair.get('attack_card', {}).get('user_id') == player_id and
                'card' in new_pair.get('attack_card', {})):
                return {'type': 'attack', 'move': new_pair['attack_card']['card']}
        elif any('defend_card' in pair for pair in curr_table) and not any('defend_card' in pair for pair in prev_table):
            # Новая карта защиты
            for pair in curr_table:
                if (pair.get('defend_card', {}).get('user_id') == player_id and
                    'card' in pair.get('attack_card', {}) and
                    'card' in pair.get('defend_card', {})):
                    return {
                        'type': 'defend',
                        'move': [pair['attack_card']['card'], pair['defend_card']['card']]
                    }
        elif prev_data['players'][0].get('state') != curr_data['players'][0].get('state'):
            # Изменение состояния
            new_state = curr_data['players'][0].get('state')
            if new_state in ['bat', 'pass', 'take']:
                return {'type': 'state', 'state': new_state, 'move': None}

        return {'type': 'wait', 'move': None}
    except Exception:
        return None

# 2. Кастомный датасет
class DurakDataset(Dataset):
    def __init__(self, dataset, cache_file="durak_dataset.pkl"):
        start_time = time.time()
        self.cache_file = cache_file

        # Проверяем, существует ли кэш
        if os.path.exists(cache_file):
            print(f"Loading cached dataset from {cache_file}...")
            with open(cache_file, 'rb') as f:
                cached_data = pickle.load(f)
            self.vectors = cached_data['vectors']
            self.actions = cached_data['actions']
            print(f"Loaded {len(self.vectors)} valid snapshots from cache")
        else:
            print("No cache found, processing dataset...")
            self.dataset = dataset['train']
            self.vectors = []
            self.actions = []

            # Парсим снимки и группируем по game_id
            parsed_snaps = []
            for s in tqdm(self.dataset, desc="Parsing snapshots"):
                try:
                    parsed = json.loads(s['snapshot'])
                    parsed_snaps.append((s, parsed))
                except Exception:
                    continue

            game_snaps_dict = {}
            for snap, parsed in parsed_snaps:
                game_id = snap['game_id']
                if game_id not in game_snaps_dict:
                    game_snaps_dict[game_id] = []
                game_snaps_dict[game_id].append((snap, parsed))

            print(f"Found {len(game_snaps_dict)} games")
            for game_id in tqdm(game_snaps_dict.keys(), desc="Processing games"):
                game_snaps = sorted(game_snaps_dict[game_id], key=lambda x: x[1].get('timestamp', 0))
                for i in range(len(game_snaps) - 1):
                    curr_snap, curr_parsed = game_snaps[i]
                    next_snap, next_parsed = game_snaps[i + 1]
                    # Проверяем входной вектор
                    input_vector = snapshot_to_input(curr_parsed)
                    if input_vector is None:
                        continue
                    # Извлекаем действие
                    action = extract_action(curr_parsed, next_parsed)
                    if (action is None or not isinstance(action, dict) or
                        'type' not in action or 'move' not in action or
                        action['type'] not in ['attack', 'defend', 'state', 'wait']):
                        continue
                    self.vectors.append(input_vector)
                    self.actions.append(action)

            print(f"Processed {len(self.vectors)} valid snapshots")
            # Сохраняем кэш
            if self.vectors:
                print(f"Saving dataset to {cache_file}...")
                with open(cache_file, 'wb') as f:
                    pickle.dump({'vectors': self.vectors, 'actions': self.actions}, f)

        if not self.vectors:
            raise ValueError("No valid snapshots available after processing")

        print(f"DurakDataset initialization took {time.time() - start_time:.3f} seconds")

    def __len__(self):
        return len(self.vectors)

    def __getitem__(self, idx):
        return torch.tensor(self.vectors[idx], dtype=torch.float32), self.actions[idx]

# 3. Модель
class DurakModel(nn.Module):
    def __init__(self, input_size=190, hidden_size=256, num_cards=24):
        super(DurakModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.action_head = nn.Linear(hidden_size, 4)  # attack, defend, state, wait
        self.attack_card_head = nn.Linear(hidden_size, num_cards)
        self.defend_card_head = nn.Linear(hidden_size, num_cards)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        action_probs = torch.softmax(self.action_head(x), dim=-1)
        attack_probs = torch.softmax(self.attack_card_head(x), dim=-1)
        defend_probs = torch.softmax(self.defend_card_head(x), dim=-1)
        return action_probs, attack_probs, defend_probs

# 4. Вспомогательная функция
def card_to_idx(card):
    """Преобразует карту в индекс (0–23)."""
    if not card or not isinstance(card, str):
        return 0
    try:
        nominal = int(card[:-1]) - 9  # 9->0, ..., 14->5
        if nominal < 0 or nominal > 5:
            return 0
        suit = {'S': 0, 'C': 1, 'D': 2, 'H': 3}[card[-1]]
        return nominal * 4 + suit
    except (ValueError, KeyError):
        return 0

# 5. Обучение
def train_model():
    start_time = time.time()
    print("Loading dataset...")
    dataset = load_dataset("neuronetties/durak")
    print(f"Dataset loaded in {time.time() - start_time:.3f} seconds")

    print("Initializing DurakDataset...")
    durak_dataset = DurakDataset(dataset)
    print("Creating DataLoader...")

    def custom_collate(batch):
        batch = [item for item in batch if item is not None]
        if not batch:
            return None
        inputs, actions = zip(*batch)
        return torch.stack(inputs), actions

    dataloader = DataLoader(durak_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate)
    print(f"DataLoader created in {time.time() - start_time:.3f} seconds")

    model = DurakModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    action_criterion = nn.CrossEntropyLoss()
    card_criterion = nn.CrossEntropyLoss()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(10):
        epoch_start_time = time.time()
        total_loss = 0
        batch_count = 0
        for batch_idx, batch in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
            if batch is None:
                continue

            inputs, actions = batch
            inputs = inputs.to(device)

            optimizer.zero_grad()
            action_probs, attack_probs, defend_probs = model(inputs)

            # Подготовка меток
            action_labels = []
            attack_labels = []
            defend_labels = []
            valid_indices = []
            for i, action in enumerate(actions):
                if not isinstance(action, dict) or action['type'] not in ['attack', 'defend', 'state', 'wait']:
                    continue
                try:
                    if action['type'] == 'attack' and action['move']:
                        action_labels.append(0)
                        attack_labels.append(card_to_idx(action['move']))
                        defend_labels.append(0)
                    elif action['type'] == 'defend' and action['move'] and len(action['move']) == 2:
                        action_labels.append(1)
                        attack_labels.append(0)
                        defend_labels.append(card_to_idx(action['move'][1]))
                    elif action['type'] == 'state':
                        action_labels.append(2)
                        attack_labels.append(0)
                        defend_labels.append(0)
                    else:  # wait
                        action_labels.append(3)
                        attack_labels.append(0)
                        defend_labels.append(0)
                    valid_indices.append(i)
                except Exception:
                    continue

            if not valid_indices:
                continue

            # Фильтруем входные данные
            inputs = inputs[valid_indices]
            action_labels = torch.tensor(action_labels, dtype=torch.long, device=device)
            attack_labels = torch.tensor(attack_labels, dtype=torch.long, device=device)
            defend_labels = torch.tensor(defend_labels, dtype=torch.long, device=device)

            action_probs, attack_probs, defend_probs = model(inputs)

            action_loss = action_criterion(action_probs, action_labels)
            attack_loss = card_criterion(attack_probs, attack_labels)
            defend_loss = card_criterion(defend_probs, defend_labels)

            loss = action_loss + attack_loss + defend_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1
            if batch_idx % 100 == 0:
                print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

        if batch_count > 0:
            print(f"Epoch {epoch+1} completed in {time.time() - epoch_start_time:.3f} seconds")
            print(f"Average Loss: {total_loss / batch_count:.4f}")
        else:
            print(f"Epoch {epoch+1} skipped: no valid batches")

    print(f"Training completed in {time.time() - start_time:.3f} seconds")
    return model

# Запуск
if __name__ == "__main__":
    # Удалите старый кэш для пересоздания
    # os.remove("durak_dataset.pkl")
    train_model()

Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/292962 [00:00<?, ? examples/s]

...

Batch 8800, Loss: 5.7041


Epoch 10: 100%|█████████▉| 8915/8958 [01:41<00:00, 90.98it/s]

Batch 8900, Loss: 5.8918


Epoch 10: 100%|██████████| 8958/8958 [01:41<00:00, 88.04it/s]

Epoch 10 completed in 101.755 seconds
Average Loss: 5.6542
Training completed in 972.950 seconds
